In [1]:
import numpy as np
import os
import time
from concurrent.futures import ProcessPoolExecutor
from typing import List, NoReturn

import h5py
import librosa
import musdb
from tqdm import tqdm


In [2]:
def float32_to_int16(x: np.float32) -> np.int16:

    x = np.clip(x, a_min=-1, a_max=1)

    return (x * 32767.0).astype(np.int16)

In [3]:
def preprocess_audio(
    audio: np.array, mono: bool, origin_sr: float, sr: float, resample_type: str
) -> np.array:
    r"""Preprocess audio to mono / stereo, and resample.

    Args:
        audio: (channels_num, audio_samples), input audio
        mono: bool
        origin_sr: float, original sample rate
        sr: float, target sample rate
        resample_type: str, e.g., 'kaiser_fast'

    Returns:
        output: ndarray, output audio
    """
    if mono:
        audio = np.mean(audio, axis=0)
        # (audio_samples,)

    output = librosa.core.resample(
        audio, orig_sr=origin_sr, target_sr=sr, res_type=resample_type
    )
    # (audio_samples,) | (channels_num, audio_samples)

    if output.ndim == 1:
        output = output[None, :]
        # (1, audio_samples,)

    return output

In [4]:
def write_single_audio_to_hdf5(param: List) -> NoReturn:
    r"""Write single audio into hdf5 file."""
    (
        dataset_dir,
        subset,
        split,
        track_index,
        source_types,
        mono,
        sample_rate,
        resample_type,
        hdf5s_dir,
    ) = param

    # Dataset of corresponding subset and split.
    mus = musdb.DB(root=dataset_dir, subsets=[subset], split=split)
    track = mus.tracks[track_index]

    # Path to write out hdf5 file.
    hdf5_path = os.path.join(hdf5s_dir, "{}.h5".format(track.name))

    with h5py.File(hdf5_path, "w") as hf:

        hf.attrs.create("audio_name", data=track.name.encode(), dtype="S100")
        hf.attrs.create("sample_rate", data=sample_rate, dtype=np.int32)

        for source_type in source_types:

            audio = track.targets[source_type].audio.T
            # (channels_num, audio_samples)

            # Preprocess audio to mono / stereo, and resample.
            audio = preprocess_audio(
                audio, mono, track.rate, sample_rate, resample_type
            )
            # (channels_num, audio_samples) | (audio_samples,)

            hf.create_dataset(
                name=source_type, data=float32_to_int16(audio), dtype=np.int16
            )

        # Mixture
        audio = track.audio.T
        # (channels_num, audio_samples)

        # Preprocess audio to mono / stereo, and resample.
        audio = preprocess_audio(audio, mono, track.rate, sample_rate, resample_type)
        # (channels_num, audio_samples)

        hf.create_dataset(name="mixture", data=float32_to_int16(audio), dtype=np.int16)

    print("{} Write to {}, {}".format(track_index, hdf5_path, audio.shape))

In [5]:
SOURCE_TYPES = ["vocals", "drums", "bass", "other", "accompaniment"]

In [6]:
SAMPLE_RATE=44100
CHANNELS=2

dataset_dir = "../datasets/musdb18"
subset="train"
split=""
hdf5s_dir=f"hdf5s/musdb18/sr={SAMPLE_RATE},chn={CHANNELS}/train"
sample_rate=SAMPLE_RATE
channels = CHANNELS


mono = True if channels == 1 else False
source_types = SOURCE_TYPES
resample_type = "kaiser_fast"

# Paths
os.makedirs(hdf5s_dir, exist_ok=True)

# Dataset of corresponding subset and split.
mus = musdb.DB(root=dataset_dir, subsets=[subset], split=split)
print("Subset: {}, Split: {}, Total pieces: {}".format(subset, split, len(mus)))

params = []  # A list of params for multiple processing.

for track_index in range(len(mus.tracks)):

    param = (
        dataset_dir,
        subset,
        split,
        track_index,
        source_types,
        mono,
        sample_rate,
        resample_type,
        hdf5s_dir,
    )

    params.append(param)

# Uncomment for debug.
# write_single_audio_to_hdf5(params[0])
# os._exit(0)

pack_hdf5s_time = time.time()

# with ProcessPoolExecutor(max_workers=None) as pool:
#     # Maximum works on the machine
#     pool.map(write_single_audio_to_hdf5, params)

with ProcessPoolExecutor(max_workers=None) as pool:
    # Initialize tqdm progress bar with total equal to the length of params
    for _ in tqdm(pool.map(write_single_audio_to_hdf5, params), total=len(params)):
        pass

print("Pack hdf5 time: {:.3f} s".format(time.time() - pack_hdf5s_time))

Subset: train, Split: , Total pieces: 100


  0%|          | 0/100 [00:00<?, ?it/s]

14 Write to hdf5s/musdb18/sr=44100,chn=2/train/BigTroubles - Phantom.h5, (2, 6471680)
5 Write to hdf5s/musdb18/sr=44100,chn=2/train/Actions - One Minute Smile.h5, (2, 7204864)
0 Write to hdf5s/musdb18/sr=44100,chn=2/train/A Classic Education - NightOwl.h5, (2, 7552000)


  1%|          | 1/100 [00:37<1:01:13, 37.11s/it]

3 Write to hdf5s/musdb18/sr=44100,chn=2/train/ANiMAL - Rockshow.h5, (2, 7299072)
6 Write to hdf5s/musdb18/sr=44100,chn=2/train/Actions - South Of The Water.h5, (2, 7788544)
4 Write to hdf5s/musdb18/sr=44100,chn=2/train/Actions - Devil's Words.h5, (2, 8671232)
7 Write to hdf5s/musdb18/sr=44100,chn=2/train/Aimee Norwich - Child.h5, (2, 8338432)
2 Write to hdf5s/musdb18/sr=44100,chn=2/train/ANiMAL - Easy Tiger.h5, (2, 9061376)
12 Write to hdf5s/musdb18/sr=44100,chn=2/train/Auctioneer - Our Future Faces.h5, (2, 9159680)
10 Write to hdf5s/musdb18/sr=44100,chn=2/train/Angela Thomas Wade - Milk Cow Blues.h5, (2, 9300992)
1 Write to hdf5s/musdb18/sr=44100,chn=2/train/ANiMAL - Clinic A.h5, (2, 10489856)


...

92 Write to hdf5s/musdb18/sr=44100,chn=2/train/Traffic Experiment - Sirens.h5, (2, 18578432)


 93%|█████████▎| 93/100 [05:07<00:28,  4.02s/it]

99 Write to hdf5s/musdb18/sr=44100,chn=2/train/Young Griffo - Pennies.h5, (2, 12251136)


100%|██████████| 100/100 [05:08<00:00,  3.09s/it]

Pack hdf5 time: 308.941 s
